In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
from scipy.signal import find_peaks
from sklearn.neighbors import KernelDensity

In [13]:
def find_most_prominent_peak(peaks,peaks_prominences,valleys,valleys_prominences,min_threshold = 0,max_treshold = inf):
    max_prominence = 0
    
    max_peak = 0
    offset = (min_threshold + max_treshold) / 2
    for peak, prominence in zip(peaks, peaks_prominences):
        if peak > max_treshold:
            continue
        if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
            max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
            max_peak = peak

    for peak, prominence in zip(valleys, valleys_prominences):
        if peak > max_treshold:
            continue
        if peak > min_threshold and prominence * calc_prominence_factor(peak,offset=offset) > max_prominence:
            max_prominence = prominence * calc_prominence_factor(peak,offset=offset)
            max_peak = peak

    return (max_peak, max_prominence)
    
def calc_prominence_factor(x,threshold = 0.2,offset=0):
    return 1 # exp(0-100*(x-offset)**2)

In [18]:
class Analysis:
    def __init__(self,location):
        self.gyro = pd.read_csv(location + "/android.sensor.gyroscope.csv")
        self.accel = pd.read_csv(location + "/android.sensor.accelerometer.csv")
        self.beats = pd.read_csv(location + "/out.csv")
    
    def preanalysis(self,prominence=(None,None)):
        self.accel_time = self.accel.iloc[:,0] / 1000000000
        self.gyro_time = self.gyro.iloc[:,0] / 1000000000

        self.accel_peaks_x_samples , temp = find_peaks(self.accel.iloc[:,1],prominence=prominence)
        self.accel_peaks_x = self.accel_time[self.accel_peaks_x_samples]
        self.accel_peaks_x_prominence = temp["prominences"]

        self.accel_valleys_x_samples , temp = find_peaks(-1*self.accel.iloc[:,1],prominence=prominence)
        self.accel_valleys_x_prominence = temp["prominences"]
        self.accel_valleys_x = self.accel_time[self.accel_valleys_x_samples]

        self.accel_peaks_y_samples , temp = find_peaks(self.accel.iloc[:,2],prominence=prominence)
        self.accel_peaks_y_prominence = temp["prominences"]
        self.accel_peaks_y = self.accel_time[self.accel_peaks_y_samples]

        self.accel_valleys_y_samples , temp = find_peaks(-1*self.accel.iloc[:,2],prominence=prominence)
        self.accel_valleys_y_prominence = temp["prominences"]
        self.accel_valleys_y = self.accel_time[self.accel_valleys_y_samples]

        self.accel_peaks_z_samples , temp = find_peaks(self.accel.iloc[:,3],prominence=prominence)
        self.accel_peaks_z_prominence = temp["prominences"]
        self.accel_peaks_z = self.accel_time[self.accel_peaks_z_samples]

        self.accel_valleys_z_samples , temp = find_peaks(-1*self.accel.iloc[:,3],prominence=prominence)
        self.accel_valleys_z_prominence = temp["prominences"]
        self.accel_valleys_z = self.accel_time[self.accel_valleys_z_samples]

        self.gyro_peaks_x_samples , temp = find_peaks(self.gyro.iloc[:,1],prominence=prominence)
        self.gyro_peaks_x_prominence = temp["prominences"]
        self.gyro_peaks_x = self.gyro_time[self.gyro_peaks_x_samples]

        self.gyro_valleys_x_samples , temp = find_peaks(-1*self.gyro.iloc[:,1],prominence=prominence)
        self.gyro_valleys_x_prominence = temp["prominences"]
        self.gyro_valleys_x = self.gyro_time[self.gyro_valleys_x_samples]

        self.gyro_peaks_y_samples , temp = find_peaks(self.gyro.iloc[:,2],prominence=prominence)
        self.gyro_peaks_y_prominence = temp["prominences"]
        self.gyro_peaks_y = self.gyro_time[self.gyro_peaks_y_samples]

        self.gyro_valleys_y_samples , temp = find_peaks(-1*self.gyro.iloc[:,2],prominence=prominence)
        self.gyro_valleys_y_prominence = temp["prominences"]
        self.gyro_valleys_y = self.gyro_time[self.gyro_valleys_y_samples]

        self.gyro_peaks_z_samples , temp = find_peaks(self.gyro.iloc[:,3],prominence=prominence)
        self.gyro_peaks_z_prominence = temp["prominences"]
        self.gyro_peaks_z = self.gyro_time[self.gyro_peaks_z_samples]

        self.gyro_valleys_z_samples , temp = find_peaks(-1*self.gyro.iloc[:,3],prominence=prominence)
        self.gyro_valleys_z_prominence = temp["prominences"]
        self.gyro_valleys_z = self.gyro_time[self.gyro_valleys_z_samples]

    def analysis_data(self):
        min_threshold = 0
        max_threshold = inf

        self.beat_diff = (self.beats.iloc[:,0][0] + self.beats.iloc[:,0][len(self.beats.iloc[:,0])-1])/ (len(self.beats.iloc[:,0])-1)
        threshold = self.beat_diff / 2

        self.accel_peaks = []
        self.accel_peaks_axis = []
        self.accel_prominences = []
        gyro_peaks = []

        for beat, i in zip(self.beats.iloc[:,0],range(len(self.beats.iloc[:,0]))):

            #if i < (len(beats)-1):
            #    max_threshold = (beats[i+1] + beat) / 2
            #else: 
            #    max_threshold = inf

            min_threshold = beat - threshold
            max_threshold = beat + threshold

            accel_peaks_with_prominence = []
            gyro_peaks_with_prominence = []


            
            accel_peaks_with_prominence.append(find_most_prominent_peak(self.accel_peaks_x,self.accel_peaks_x_prominence,self.accel_valleys_x,self.accel_valleys_x_prominence,min_threshold, max_threshold))
            accel_peaks_with_prominence.append(find_most_prominent_peak(self.accel_peaks_y,self.accel_peaks_y_prominence,self.accel_valleys_y,self.accel_valleys_y_prominence,min_threshold, max_threshold))
            accel_peaks_with_prominence.append(find_most_prominent_peak(self.accel_peaks_z,self.accel_peaks_z_prominence,self.accel_valleys_z,self.accel_valleys_z_prominence,min_threshold, max_threshold))

            gyro_peaks_with_prominence.append(find_most_prominent_peak(self.gyro_peaks_x,self.gyro_peaks_x_prominence,self.gyro_valleys_x,self.gyro_valleys_x_prominence,min_threshold, max_threshold))
            gyro_peaks_with_prominence.append(find_most_prominent_peak(self.gyro_peaks_y,self.gyro_peaks_y_prominence,self.gyro_valleys_y,self.gyro_valleys_y_prominence,min_threshold, max_threshold))
            gyro_peaks_with_prominence.append(find_most_prominent_peak(self.gyro_peaks_z,self.gyro_peaks_z_prominence,self.gyro_valleys_z,self.gyro_valleys_z_prominence,min_threshold, max_threshold))

            max_accel_prominence = 0
            max_peak_position = 0
            for peak, prominence in accel_peaks_with_prominence:
                if prominence > max_accel_prominence:
                    max_accel_prominence = prominence
                    max_peak_position = peak

            accel_peak_sum = 0
            accel_peak_count = 0
            axis = []


            
            for (peak, prominence), index in zip(accel_peaks_with_prominence,range(3)):
                if prominence  >= (max_accel_prominence * 0.50) and abs(peak-max_peak_position) < threshold / 4:
                    accel_peak_sum += peak
                    accel_peak_count += 1
                    axis.append(index)
                    #self.accel_prominences.append(prominence)
            if accel_peak_sum > 0:
                accel_peak = accel_peak_sum / accel_peak_count
                self.accel_peaks.append((accel_peak,max_accel_prominence,accel_peak_count,beat))
                self.accel_peaks_axis.append(axis)
                self.accel_prominences.append(max_accel_prominence)
            else:
                self.accel_peaks.append((None,None,None,beat))
                self.accel_peaks_axis.append([])    
            min_threshold = max_threshold


    def plot_acceleration_preprosesed(self):
        plt.close()
        fig, axs = plt.subplots(ncols = 1, nrows = 3,sharey=True,sharex=True)

        fig.suptitle('acceleration with peaks with prominence > 25', fontsize=16)

        fig.supxlabel("time in seconds")
        fig.supylabel("acceleration in m/s^2")

        for x, y in zip(self.beats.iloc[:,0], self.beats.iloc[:,1]):

            seconds = x#+ offset_seconds

            if (y == 1):
                linestyle = 'solid'
            else:
                linestyle = 'dotted'
            axs[0].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[1].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[2].axvline(seconds, color = 'g',linestyle = linestyle)


        axs[0].plot(self.accel_peaks_x, np.array(self.accel.iloc[self.accel_peaks_x_samples,1]), "x")
        axs[0].plot(self.accel_valleys_x, np.array(self.accel.iloc[self.accel_valleys_x_samples,1]), "o")
        axs[0].plot(self.accel_time,self.accel.iloc[:,1], color = 'r')
        axs[0].grid(True)
        axs[0].set_title("x")


        axs[1].plot(self.accel_peaks_y, np.array(self.accel.iloc[self.accel_peaks_y_samples,2]), "x")
        axs[1].plot(self.accel_valleys_y, np.array(self.accel.iloc[self.accel_valleys_y_samples,2]), "o")
        axs[1].plot(self.accel_time,self.accel.iloc[:,2], color = 'r')
        axs[1].grid(True)
        axs[1].set_title("y")


        axs[2].plot(self.accel_peaks_z, np.array(self.accel.iloc[self.accel_peaks_z_samples,3]), "x")
        axs[2].plot(self.accel_valleys_z, np.array(self.accel.iloc[self.accel_valleys_z_samples,3]), "o")
        axs[2].plot(self.accel_time,self.accel.iloc[:,3], color = 'r')
        axs[2].grid(True)
        axs[2].set_title("z")


        plt.grid(True)
        plt.show()

    def plot_gyro_preprosesed(self):
        plt.close()
        fig, axs = plt.subplots(ncols = 1, nrows = 3,sharey=True,sharex=True)

        fig.suptitle('gyro with peaks with prominence > 25', fontsize=16)

        fig.supxlabel("time in seconds")
        fig.supylabel("gyro")

        for x, y in zip(self.beats.iloc[:,0], self.beats.iloc[:,1]):

            seconds = x#+ offset_seconds

            if (y == 1):
                linestyle = 'solid'
            else:
                linestyle = 'dotted'
            axs[0].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[1].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[2].axvline(seconds, color = 'g',linestyle = linestyle)


        axs[0].plot(self.gyro_peaks_x, np.array(self.gyro.iloc[self.gyro_peaks_x_samples,1]), "x")
        axs[0].plot(self.gyro_valleys_x, np.array(self.gyro.iloc[self.gyro_valleys_x_samples,1]), "o")
        axs[0].plot(self.gyro_time,self.gyro.iloc[:,1], color = 'r')
        axs[0].grid(True)
        axs[0].set_title("x")


        axs[1].plot(self.gyro_peaks_y, np.array(self.gyro.iloc[self.gyro_peaks_y_samples,2]), "x")
        axs[1].plot(self.gyro_valleys_y, np.array(self.gyro.iloc[self.gyro_valleys_y_samples,2]), "o")
        axs[1].plot(self.gyro_time,self.gyro.iloc[:,2], color = 'r')
        axs[1].grid(True)
        axs[1].set_title("y")


        axs[2].plot(self.gyro_peaks_z, np.array(self.gyro.iloc[self.gyro_peaks_z_samples,3]), "x")
        axs[2].plot(self.gyro_valleys_z, np.array(self.gyro.iloc[self.gyro_valleys_z_samples,3]), "o")
        axs[2].plot(self.gyro_time,self.gyro.iloc[:,3], color = 'r')
        axs[2].grid(True)
        axs[2].set_title("z")


        plt.grid(True)
        plt.show()

    def plot_beat_diff(self):
        self.calculate_running_average()
        #plt.close()
        plt.axhline(1)
        #plt.axhline(threshold)
        plt.xlabel("time in seconds")
        plt.ylabel("distance to next beat in seconds")
        plt.title("time delta for each beat")
        #plt.axhline(-1*beat_diff)
        #plt.axhline(-1*threshold)
        label = True
        for peak, prominence , _ ,beat in self.accel_peaks:
            if (peak == None):
                continue
            diff = abs(peak - beat)
            if label:
                plt.plot(beat, diff/self.beat_diff,"b.", label="beat")
                label = False
            else:
                plt.plot(beat, diff/self.beat_diff,"b.") 


        for averages, beats in self.running_average:

            plt.plot(beats,averages/self.beat_diff,"r")

        plt.grid(True)
        plt.legend()
        plt.show()

    def plot_accelration_with_motion_beat(self):
        plt.close()
        fig, axs = plt.subplots(ncols = 1, nrows = 3,sharey=True,sharex=True)

        fig.suptitle('acceleration with motion- and music beats', fontsize=16)

        fig.supxlabel("time in seconds")
        fig.supylabel("acceleration in m/s^2")


        axs[2].axvline(self.beats.iloc[:,0][0], color = 'g',label = "music beat")


        for peak, axis in zip(self.accel_peaks, self.accel_peaks_axis):
            if 2 in axis:
                axs[2].axvline(peak[0],color='b',label = "motion beat")
                break
        axs[2].legend()
                
        #axs[accel_peaks_axis[0][0]].axvline(accel_peaks[0][0],color='b',label = "b")


        for x, y in zip(self.beats.iloc[:,0],self.beats.iloc[:,1]):
            seconds = x

            if (y == 1):
                linestyle = 'solid'
            else:
                linestyle = 'dotted'
            axs[0].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[1].axvline(seconds, color = 'g',linestyle = linestyle)
            axs[2].axvline(seconds, color = 'g',linestyle = linestyle)

        for peak, axis in zip(self.accel_peaks, self.accel_peaks_axis):
            for i in axis:
                axs[i].axvline(peak[0], color = 'b')


        axs[0].plot(self.accel_peaks_x, np.array(self.accel.iloc[self.accel_peaks_x_samples,1]), "x")
        axs[0].plot(self.accel_valleys_x, np.array(self.accel.iloc[self.accel_valleys_x_samples,1]), "o")
        axs[0].plot(self.accel_time,self.accel.iloc[:,1], color = 'r')
        axs[0].grid(True)
        axs[0].set_title("x")


        axs[1].plot(self.accel_peaks_y, np.array(self.accel.iloc[self.accel_peaks_y_samples,2]), "x")
        axs[1].plot(self.accel_valleys_y, np.array(self.accel.iloc[self.accel_valleys_y_samples,2]), "o")
        axs[1].plot(self.accel_time,self.accel.iloc[:,2], color = 'r')
        axs[1].grid(True)
        axs[1].set_title("y")


        axs[2].plot(self.accel_peaks_z, np.array(self.accel.iloc[self.accel_peaks_z_samples,3]), "x")
        axs[2].plot(self.accel_valleys_z, np.array(self.accel.iloc[self.accel_valleys_z_samples,3]), "o")
        axs[2].plot(self.accel_time,self.accel.iloc[:,3], color = 'r')
        axs[2].grid(True)
        axs[2].set_title("z")


        plt.grid(True)
        axs[0].legend()
        plt.show()

    def analysis(self):
        self.preanalysis()
        self.analysis_data()

        new_array= np.reshape(self.accel_prominences, (1, -1))
        #print(new_array)

        kde = KernelDensity(kernel='epanechnikov',bandwidth='silverman').fit(np.array(self.accel_prominences).reshape((-1,1)))
        x_plot = np.linspace(floor(min(self.accel_prominences)),ceil(max(self.accel_prominences)),1000)[:,np.newaxis]
        log_dens = kde.score_samples(x_plot)

        peaks , _ = find_peaks(-1 * np.exp(log_dens),height=-0.02)
        min_prominence= x_plot[peaks[0]][0]

        self.preanalysis(prominence=(min_prominence,None))
        self.analysis_data()

        plt.plot(x_plot[:,0],np.exp(log_dens))
        plt.plot(x_plot[peaks],np.exp(log_dens)[peaks],"x")
        plt.grid(True)
        plt.show()
    

    def calculate_running_average(self,bandwith=10):
        averages = []
        beats = []
        elements = []
        self.running_average = []
        for peak, _ , _ ,beat in self.accel_peaks:
            if peak == None:
                elements = []
                if len(averages)>0:
                    self.running_average.append((averages,beats))
                    averages = []
                    beats = []
                continue
            elements.append(abs(peak - beat))
            if len(elements) > bandwith:
                elements = elements[1:]
            if len(elements) > bandwith / 2:
                average = sum(elements)/ len(elements)
                averages.append(average)
                beats.append(beat)
        if len(averages)>0:
            self.running_average.append((averages,beats))

    def shift_beat(self, factor):
        print(self.beats)
        self.beats = self.beats + self.beat_diff*factor
        print(self.beats)

lotta = Analysis("donnerstag_extratraining/Lotta/")
#lotta.analysis()
#lotta.shift_beat(0.5)
lotta.analysis()
plt.close()
lotta.plot_beat_diff()

In [86]:
phil = Analysis("donnerstag_extratraining/Phil/")
phil.analysis()

[ 21  43  50  56  62  73  81  89  98 105 116 127 136 147 160 174 192 221
 241 252 267 283 298 314 327 337 339 343 348 359 367 374 381 409 440 473
 510 536 555 557 563 582 600 615 634 646 655 667 693 723 766 805 813 835
 864 916 961 970 985]
2.5855855855855854


In [13]:
phil.plot_acceleration_preprosesed()

In [78]:
phil.plot_accelration_with_motion_beat()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [116]:
phil.plot_beat_diff()

In [27]:
phil.plot_gyro_preprosesed()

In [124]:
lotta = Analysis("donnerstag_extratraining/Lotta/")
lotta.analysis()

[ 29  36  43  53  58  60  66  76  78  87  99 108 116 120 126 130 132 134
 138 142 147 154 162 173 181 191 195 198 200 206 217 228 231 237 242 248
 250 252 258 262 266 272 279 282 287 296 301 306 310 314 319 321 326 329
 337 343 354 367 373 379 385 389 399 410 431 452 468 482 495 538 634 760
 907]
4.18018018018018


In [19]:
lotta.plot_acceleration_preprosesed()

In [20]:
lotta.plot_accelration_with_motion_beat()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [16]:
lotta = Analysis("donnerstag_extratraining/Lotta/")
lotta.analysis()
plt.close()
lotta.plot_beat_diff()

[ 29  36  43  53  58  60  66  76  78  87  99 108 116 120 126 130 132 134
 138 142 147 154 162 173 181 191 195 198 200 206 217 228 231 237 242 248
 250 252 258 262 266 272 279 282 287 296 301 306 310 314 319 321 326 329
 337 343 354 367 373 379 385 389 399 410 431 452 468 482 495 538 634 760
 907]
4.18018018018018


In [29]:
lotta.plot_gyro_preprosesed()

Shows the acceleration over time split by axis and with peaks with a prominence over 25

Shows the time diffrent each motionbeat has to its next musicbeat